In [1]:
from acc2sf_offline import get_cadence
from media_player import Player
import numpy as np
import time
import pandas as pd
# from collections import namedtuple
from os import getenv
from typing import List, NamedTuple
from dotenv import load_dotenv
import logging
load_dotenv(dotenv_path='../.env')
#from HR_2_optimal_HR import *



True

In [2]:
# Sample = namedtuple(typename='Sample', field_names=['hr', 'cadance'])

class Sample(NamedTuple):
    hr: float
    cadence: float

logger = logging.getLogger(name='controller')

def log(i: int, sample: Sample, text: str):
    """
    Logging with severity level `INFO`
    """
    logger.info(f'{i}: {text}   {sample=}')

def log_error(text: str):
    """
    Logging with severity level `ERROR`
    """
    logger.error(text)

def log_up(i: int, sample: Sample):
    log(i=i, sample=sample, text='temp up')

def log_down(i: int, sample: Sample):
    log(i=i, sample=sample, text='temp down')

def log_unchanged(i: int, sample: Sample):
    log(i=i, sample=sample, text='temp unchanged')



In [3]:
# This module adapts music tempo to HR by the following rules:
# current HR & cadnece above/below optimal  -> speed up/down tempo
# every stabilization_period, a decision for changing the tempo will be taken
# this modules ignores warm up period


THRESHOLD_HR = int(getenv('THRESHOLD_HR'), 10) # In percentages
"""
Threashold for HR divergance from oprimal HR
* Units in precentages
"""
THRESHOLD_CADANCE = int(getenv('THRESHOLD_CADANCE'), 10)# In percentages
"""
Threashold for cadence divergance from song tempo
* Units in precentages
"""
INTERVAL = int(getenv('INTERVAL'), 10)
"""
Sensor window time
* Units in seconds
* Must be devisible by `10` (???)
"""
STABLE_COUNTDOWN = int(getenv('STABLE_COUNTDOWN'), 3)
"""
Number of interval with time `INTERVAL` to allow for HR stabilization
"""
# initiate audio player
player = Player(['../recordings/running_track_sunshine_jaunt.mp3'])

# module initialization
# delta = 10
#warm_up_time = 100 # in sec
runner_age = 26
runner_max_HR = 220 - runner_age # TODO: find more accurate formula
low_HR, high_HR = 95.95, 134.33
def get_optimal_heart_rate():
    return np.mean([low_HR, high_HR]).round(decimals=2)
 
# Start of Warming Up period
# time.sleep(warm_up_time)
# optimal_HR = HR_to_optimal_HR(low_HR, high_HR)
# End of Warming Up period

# averaging cadence and HR in spans of 15 sec
cadence = get_cadence("../recordings/gym_12kmh/ACC.csv", running=True) # assuming cadence is measured after warm up period
print(f'{np.count_nonzero(cadence)=}')
cadence = cadence[:len(cadence)-len(cadence)%INTERVAL]
cadence = cadence.reshape(-1, INTERVAL)
cadence = np.mean(cadence, axis=1)

df = pd.read_csv("../recordings/gym_12kmh/HR.csv", skiprows=2, header=None, names=['hr']) #  the signal is in 1Hz
heart_rate_signal = df['hr'].to_numpy()
heart_rate_signal = heart_rate_signal[:len(heart_rate_signal)-len(heart_rate_signal)%INTERVAL]
heart_rate_signal = heart_rate_signal.reshape(-1, INTERVAL)
heart_rate_signal = np.mean(heart_rate_signal, axis=1)
least_length = min(len(heart_rate_signal), len(cadence))
heart_rate_signal = heart_rate_signal[:least_length].round(decimals=2)
cadence = cadence[:least_length].round(decimals=2)
print(f'{cadence=}')
print(f'{heart_rate_signal=}')

samples = [Sample(*s) for s in zip(heart_rate_signal, cadence)]
logger.info(f'First song tempo: {player.current_song.tempo}')
for i, sample in enumerate(samples):
    hr, cadence = sample
    # stable_countdown = 
    optimal_hr = get_optimal_heart_rate()
    hr_divergence = 100*(hr-optimal_hr)/optimal_hr
    tempo = player.current_song.tempo
    cadence_divergence = 100*(cadence*60-tempo)/tempo # beats per minute
    if(hr_divergence > THRESHOLD_HR):
        if(cadence_divergence > THRESHOLD_CADANCE):
            log_down(i, sample)
        
        print(f'{sample=}: tempo down')
    elif(hr < -THRESHOLD_HR):
        print(f'{sample=}: tempo up')
    else:
        print(f'{sample=}: tempo unchanged')
    # stabilization period
    # time.sleep(stabilization_period)

np.count_nonzero(cadence)=223
cadence=array([1.85, 2.29, 2.6 , 2.57, 2.29, 1.23, 1.79, 2.26, 2.58, 2.31, 2.59,
       2.6 , 1.52, 1.81, 2.34, 2.32, 2.31, 2.25, 1.49, 2.31, 2.62, 2.65,
       2.74, 2.64, 0.  , 1.07])
heart_rate_signal=array([ 83.9 , 125.23, 137.07, 141.91, 144.35, 146.01, 152.36, 153.82,
       153.9 , 154.02, 154.28, 154.61, 154.56, 154.9 , 155.33, 155.67,
       155.95, 155.65, 155.8 , 155.89, 155.93, 156.27, 156.94, 158.62,
       153.95, 141.5 ])
sample=Sample(hr=83.9, cadance=1.85): tempo unchanged
sample=Sample(hr=125.23, cadance=2.29): tempo unchanged
sample=Sample(hr=137.07, cadance=2.6): tempo down
sample=Sample(hr=141.91, cadance=2.57): tempo down
sample=Sample(hr=144.35, cadance=2.29): tempo down
sample=Sample(hr=146.01, cadance=1.23): tempo down
sample=Sample(hr=152.36, cadance=1.79): tempo down
sample=Sample(hr=153.82, cadance=2.26): tempo down
sample=Sample(hr=153.9, cadance=2.58): tempo down
sample=Sample(hr=154.02, cadance=2.31): tempo down
sample=Sample